# Getting Started with RasterFrames Notebook

## Setup Spark Environment

In [1]:
from pyspark.sql import SparkSession
from pyrasterframes import *
from pyrasterframes.rasterfunctions import *

spark = SparkSession.builder \
    .withKryoSerialization() \
    .getOrCreate() \
    .withRasterFrames()

In [3]:
from pyrasterframes import RFContext
RFContext.active().build_info()

'{"builtAtMillis":"1557330142297", "name":"core", "rfSparkVersion":"2.3.2", "moduleName":"rasterframes", "scalaVersion":"2.11.12", "rfGeoTrellisVersion":"2.2.0", "version":"0.8.0-SNAPSHOT", "sbtVersion":"1.2.8", "builtAtString":"2019-05-08 15:42:22.297", "rfGeoMesaVersion":"2.2.1"}'

## Read an image

In [4]:
href = "https://modis-pds.s3.amazonaws.com/MCD43A4.006/31/11/2017158/MCD43A4.A2017158.h31v11.006.2017171203421_B01.TIF"
df = spark.read.rastersource(href)

In [5]:
df.printSchema()

root
 |-- tile_path: string (nullable = false)
 |-- tile: struct (nullable = true)
 |    |-- tileContext: struct (nullable = false)
 |    |    |-- extent: struct (nullable = false)
 |    |    |    |-- xmin: double (nullable = false)
 |    |    |    |-- ymin: double (nullable = false)
 |    |    |    |-- xmax: double (nullable = false)
 |    |    |    |-- ymax: double (nullable = false)
 |    |    |-- crs: struct (nullable = false)
 |    |    |    |-- crsProj4: string (nullable = false)
 |    |-- tile: tile (nullable = false)



In [12]:
df.select(rf_extent("tile"), rf_crs("tile")).toPandas()

,rf_extent(tile),crs(tile)
0,"(14455356.755667, -2342509.0947641465, 1457396...",(+proj=sinu +lon_0=0.0 +x_0=0.0 +y_0=0.0 +a=63...
1,"(14573964.81109804, -2342509.0947641465, 14692...",(+proj=sinu +lon_0=0.0 +x_0=0.0 +y_0=0.0 +a=63...
2,"(14692572.86652908, -2342509.0947641465, 14811...",(+proj=sinu +lon_0=0.0 +x_0=0.0 +y_0=0.0 +a=63...
3,"(14811180.92196012, -2342509.0947641465, 14929...",(+proj=sinu +lon_0=0.0 +x_0=0.0 +y_0=0.0 +a=63...
4,"(14929788.97739116, -2342509.0947641465, 15048...",(+proj=sinu +lon_0=0.0 +x_0=0.0 +y_0=0.0 +a=63...
5,"(15048397.0328222, -2342509.0947641465, 151670...",(+proj=sinu +lon_0=0.0 +x_0=0.0 +y_0=0.0 +a=63...
6,"(15167005.08825324, -2342509.0947641465, 15285...",(+proj=sinu +lon_0=0.0 +x_0=0.0 +y_0=0.0 +a=63...
7,"(15285613.14368428, -2342509.0947641465, 15404...",(+proj=sinu +lon_0=0.0 +x_0=0.0 +y_0=0.0 +a=63...
8,"(15404221.199115321, -2342509.0947641465, 1552...",(+proj=sinu +lon_0=0.0 +x_0=0.0 +y_0=0.0 +a=63...
9,"(15522829.254546361, -2342509.0947641465, 1556...",(+proj=sinu +lon_0=0.0 +x_0=0.0 +y_0=0.0 +a=63...


In [13]:
df.select("tile").toPandas()

,tile
0,"(((14455356.755667, -2342509.0947641465, 14573..."
1,"(((14573964.81109804, -2342509.0947641465, 146..."
2,"(((14692572.86652908, -2342509.0947641465, 148..."
3,"(((14811180.92196012, -2342509.0947641465, 149..."
4,"(((14929788.97739116, -2342509.0947641465, 150..."
5,"(((15048397.0328222, -2342509.0947641465, 1516..."
6,"(((15167005.08825324, -2342509.0947641465, 152..."
7,"(((15285613.14368428, -2342509.0947641465, 154..."
8,"(((15404221.199115321, -2342509.0947641465, 15..."
9,"(((15522829.254546361, -2342509.0947641465, 15..."
